Возьмите задачу с винами (https://www.kaggle.com/rajyellow46/wine-quality) и решите ее, оформив в виде CrispDM-подхода. Решение - jupyter notebook на github или colab.
! Только для тех у кого уже диплом ! Оформите задачи по дипломной работе в виде этапов CrispDM (например, в trello). Пришлите скриншот.

# 1. Понимание бизнеса

## 1.1 Цель
Предсказать выживание на Титанике

## 1.2 Описание
Катастрофа Титаника - одно из самых печально известных кораблекрушений в истории. 15 апреля 1912 года во время своего первого плавания «Титаник» затонул после столкновения с айсбергом, убив 1502 из 2224 пассажиров и членов экипажа. Эта сенсационная трагедия потрясла международное сообщество и привела к улучшению правил безопасности судов.

Одна из причин того, что кораблекрушение привело к такому большому количеству жертв, заключалась в том, что на корбале не хватало спасательных шлюпок для пассажиров и членов экипажа. Несмотря на то, что в выживании в этой катастрофе был элемент случая, некоторые группы людей имели больше шансов выжить, чем другие (такие как женщины, дети и представители высшего класса).

В этой задаче мы просим вас завершить анализ того, какие люди могут выжить. В частности, мы просим вас применить инструменты машинного обучения, чтобы предсказать, какие пассажиры пережили трагедию.

[Описание от Kaggle](https://www.kaggle.com/c/titanic)

# 2. Data Understanding

## 2.1 Import Libraries

## 2.2 Вспомогательные функции

## 2.3 Загрузка данных

## 2.4 Статистика и визуализации

# 3. Data Preparation

## 3.1 Категориальные переменные должны быть преобразованы в числовые переменные

Переменные Embarked, Pclass и Sex рассматриваются как категориальные переменные. Некоторые из  алгоритмов могут обрабатывать только числовые значения, поэтому нам нужно создать новую (фиктивную) переменную для каждого уникального значения категориальных переменных (OneHotEncoding)

## 3.2 Заполнить пропущенные значения в переменных
Большинство алгоритмов машинного обучения требуют, чтобы все переменные имели значения, чтобы использовать их для обучения модели. Самый простой метод - заполнить пропущенные значения средним по переменной для всех наблюдений в обучающем наборе.

## 3.3 Feature Engineering &ndash; добавляем новые признаки
http://ahmedbesbes.com/how-to-score-08134-in-titanic-kaggle-challenge.html

## 3.4 Сборка финальных датасетов для моделирования

# 4. Моделирование
Теперь мы выберем модель, которую хотели бы попробовать. Используем обучающий набор данных для обучения модели и затем проверим ее с помощью тестового набора.

## 4.1 Выбор модели
Хорошей отправной точкой является логистическая регрессия.

## 4.2 Обучение модели

# 5. Оценка
Теперь мы собираемся оценить модель

## 5.1 Модель
Мы можем оценить точность модели, используя набор для валидации, где мы знаем фактический результат. Этот набор данных не использовался для обучения, поэтому он абсолютно новый для модели.

Затем мы сравниваем точность с точностью при использовании модели на тренировочных данных. Если разница между ними значительна, это свидетельствует о переобучении. Мы стараемся избегать этого, потому что это означает, что модель не будет хорошо обобщаться на новые данные (будет работать плохо)

# 6. Развертывание

Развертывание в данном означает публикацию полученного прогноза в таблицу лидеров Kaggle.

In [ ]:
test_Y = model.predict( test_X )
passenger_id = full[891:].PassengerId
test = pd.DataFrame( { 'PassengerId': passenger_id , 'Survived': test_Y } )
test.shape
test.head()
test.to_csv( 'titanic_pred.csv' , index = False )